# Personal Milestone


## Global Fire Weather Indices - ISI using overwintered DC start-up
Dataset: initial_spread_index_1979-2018.nc
- Description: \
This dataset was developed by Natural Resources Canada using the European Centre for Medium-range Weather Forecasts (ECMWF) ERA5-HRS Reanalysis product (C3S, 2017) as inputs to the Canadian Forest Fire Danger Rating System R Package (Wang et al. 2017). \
ISI index is a numerical rating of the expected rate of firespread. It combines the effects of wind and FFMC on rate of spread without the influence of variable quantities of fuel.

- Citation: \
Megan McElhinny, Piyush Jain, Justin F. Beckers, Chelene Hanes, & Mike Flannigan. (2019). Global Fire Weather Indices - ISI using overwintered DC start-up (1.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.3540920

- [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3540920.svg)](https://doi.org/10.5281/zenodo.3540920)

In [2]:
import pandas as pd

## Database Connection
Set up and use an Azure MySQL Database to manage and share NetCDF-processed data efficiently within a group of students. This setup leverages the flexibility and scalability of Azure’s cloud services, combined with MySQL’s robust database management capabilities, to enable seamless collaboration, real-time access, and centralized data management for your project.

Azure Student Account for Cost-Free Access:
Use your Azure for Students account to access free cloud credits and services provided by Microsoft. This allows students to explore and use Azure’s platform without incurring additional costs.
- Benefits:
	- Free credits (e.g., $100 credit).
	- Access to Azure Database for MySQL with limited resource tiers suitable for small projects.
	- ntegration with other Azure services (e.g., storage, analysis, and machine learning tools).

In [12]:
# Connect to SQL Database
# Database Secret are stored in .env File
from sqlalchemy import create_engine, orm, func, text
from dotenv import  find_dotenv, load_dotenv
import os
ENV = find_dotenv('env/azure_mysql.env')
load_dotenv(ENV)
USER = os.getenv('USER_')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
DATABASE = os.getenv('DATABASE')
URI = f'mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}:3306/{DATABASE}'
engine = create_engine(URI)

try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


## Data PrePrcoess
The objective is to process and analyze climate or geospatial data stored in multiple NetCDF files spanning from 2011 to 2020, extract specific data for the province of Alberta (defined by a range of latitude and longitude), and store the resulting data in a structured database format. The detailed steps are as follows:

1.	Merge NetCDF Data (2011-2020):
- Load all NetCDF files containing geospatial or climate-related data for the years 2011 to 2020.
- 	Merge the files into a single dataset while ensuring proper alignment of time, latitude, and longitude dimensions. Handle missing data or inconsistencies, if present.
- 	Ensure that metadata, such as variable names and units, are consistent across all files.


2.	Extract Data for Alberta by Latitude and Longitude Range:
- 	Alberta’s geographical boundaries will be defined by a specific range of latitude and longitude. Filter the merged dataset to extract data that falls within this geographic range.
- 	Transform the data into a monthly time resolution, aggregating or interpolating the data as needed. This ensures the dataset represents monthly values for the selected region.

3.  Resample Data dimension From Day to Month by Average.
  
4.	Store the Processed Data in a Database:
Design a database schema to store the extracted monthly data for Alberta. This schema should include columns for:
- 	Date/Time (monthly resolution): To track the temporal component of the data.
- 	Latitude and Longitude: To retain the spatial granularity of the data.
- 	Variable Values: For each relevant variable extracted from the NetCDF files (e.g., temperature, precipitation, etc.).



In [ ]:
# Import Self-defined Class for manipulating .nc data type.

from nc_operation import nc_operation
INDEX_TYPE = 'ISI'

# 1. Merge Data From 2011 to 2020
# 2. Extract Date from Predefined Longitude and Latitude Ranges of Alberta in the Class
nco = nc_operation(file_path_pattern=f'ISI_dataset/initial_spread_index_*.nc', index_type=INDEX_TYPE, start_year=2011, end_year=2020)

In [ ]:
alberta_data = nco.data
mothly_isi = alberta_data.resample(Time='1ME').mean()
mothly_isi = mothly_isi.to_dataframe().reset_index()
mothly_isi.fillna(0, inplace=True)
mothly_isi.head()

In [ ]:
try:
    with engine.connect() as connection:
        mothly_isi.to_sql('isi_daily_data', connection, if_exists='replace',chunksize=100000, index=False)  # Use connection directly
        print("Data inserted successfully!")
except Exception as e:
    print(f"Data insertion failed: {e}")  # Print the error message
    connection.rollback()


## Initial Spread Index (ISI) Dataset Analysis
RangeIndex: 221400 entries, 0 to 221399
Data columns (total 5 columns):
| Column     | Non-Null Count | Dtype          | Range
|------------|----------------|----------------|----------------
| Latitude   | 221,400        | float64        | [49.0, 60.0 ]
| Longitude  | 221,400        | float64        | [-115, -105 ]
| Time       | 221,400        | datetime64[ns] | [2011, 2020]
| ISI        | 221,400        | float64        | [0.00, 19.10]
| crs        | 221,400        | float64        |


In [ ]:
with engine.connect() as connection:
    sql = "select * from isi_daily_data"
    df = pd.read_sql(sql,con=connection.connection)
df.head()

## Insightful Queries


##### `The Average of ISI Risk Index Value in Each Month from 2011 to 2020 in Alberta.`
- Seasonal Patterns: By analyzing monthly averages, researchers can identify high-risk wildfire months (e.g., during summer when conditions are hotter and drier) and compare trends across years.

- Yearly Comparisons: Detect whether wildfire risks are increasing or decreasing over the decade, potentially correlating this with changes in climate (e.g., rising temperatures or altered precipitation patterns).

- Resource Allocation: Helps wildfire management agencies prioritize resources during months with historically higher ISI risk.


In [ ]:
# 1. The Average of ISI Risk index value in each Month from 2011 to 2020 in Alberta.
with engine.connect() as connection:
    avg_risk_index_of_month_query = '''
    SELECT 
        EXTRACT(MONTH FROM Time) AS Month,
        AVG(IFNULL(ISI,0)) AS Avg_ISI
    FROM 
        isi_monthly_data
    GROUP BY
        Month;
    '''
    avg_risk_index_of_month_data = pd.read_sql(avg_risk_index_of_month_query,con=connection.connection, index_col='Month')
avg_risk_index_of_month_data.T


##### `The Percentage of Area with ISI Risk in Each Month from 2011 to 2020 in Alberta`
- Geographical Coverage of Risk: Understanding how wildfire risk spreads spatially across Alberta. An increasing percentage indicates a growing problem of widespread wildfire risk.

- Impact of Mitigation Efforts: Helps evaluate the effectiveness of strategies like controlled burns or forest thinning in reducing the overall area 


In [ ]:
# 2. The Percentage of Area with ISI riskness in each Month from 2011 to 2020 in Alberta 
with engine.connect() as connection:
    percentage_of_isi_risk_area_query = '''
    SELECT
        EXTRACT(YEAR_MONTH FROM Time) AS YearMonth,
        SUM(CASE WHEN ISI <> 0 THEN 1 ELSE 0 END) / COUNT(*) * 100 AS Percentage_of_Area
    FROM 
        isi_monthly_data
    GROUP BY 
        YearMonth
    '''
    percentage_of_isi_risk_area_data = pd.read_sql(percentage_of_isi_risk_area_query,con=connection.connection)
percentage_of_isi_risk_area_data

##### `The ISI High Risk Area Coordinates from 2011 to 2020 in Alberta`
According to FWI System, when ISI index over 10 represents the high riskness level of initial fire spreading.

- Hotspot Identification: Pinpoints areas that are consistently at high risk of wildfires, enabling targeted mitigation strategies like clearing dry vegetation or enhancing fire suppression infrastructure.

- Risk Zone Mapping: Helps create detailed wildfire risk maps, which are crucial for emergency planning and response.

In [ ]:
# 3. The ISI High Risk Area Coordination happen from 2011 tp 2020 in Alberta
with engine.connect() as connection:
    high_isi_risk_query = '''
    SELECT
        EXTRACT(YEAR_MONTH FROM Time) AS YearMonth,
        ISI,
        Latitude,
        Longitude
    FROM 
        isi_monthly_data
    WHERE ISI > 10
    '''
    high_isi_risk_query = pd.read_sql(high_isi_risk_query,con=connection.connection)

high_isi_risk_query

##### `Four Area Dimensions (NW, NE, SW, SE) ISI  index from 2011 to 2020 in Alberta`
The ISI (Initial Spread Index) data from 2011 to 2020 in Alberta, categorized into four dimensions (likely NW, NE, SW, SE quadrants).

- Identify High-Risk Zones: Analyze ISI trends in different regions (NW, NE, SW, SE) to identify which areas are more prone to wildfires during specific years or seasons.

- Local Hotspots: High ISI values in specific regions (e.g., NE quadrant) can point to areas where weather, fuel conditions, or topography make wildfires more likely to spread rapidly.

In [ ]:
# 4. Categorize Four Area Dimensions ISI  index from 2011 tp 2020 in Alberta
with engine.connect() as connection:
    dims_area_data_query = '''
    SELECT Time, ISI,
       CASE 
           WHEN Latitude >= 54.5 AND Longitude <= -110.0 THEN 'NW'
           WHEN Latitude >= 54.5 AND Longitude > -110.0 THEN 'NE'
           WHEN Latitude < 54.5 AND Longitude <= -110.0 THEN 'SW'
           WHEN Latitude < 54.5 AND Longitude > -110.0 THEN 'SE'
       END AS Quadrant
FROM isi_monthly_data;
    '''
    dims_area_data = pd.read_sql(dims_area_data_query,con=connection.connection)

dims_area_data

##### `Duration of ISI Transitions from Zero to Non-Zero and Back to Zero`

- Seasonal Patterns: By analyzing the duration of ISI transitions on a seasonal basis, researchers can identify periods when wildfires are more likely to persist for extended periods (e.g., late spring to early fall), helping to pinpoint high-risk wildfire seasons.

- Yearly Comparisons: Comparing the average wildfire durations across years can reveal trends such as whether wildfire events are becoming longer over time, potentially linked to climate change factors like increasing droughts or shifting weather patterns.

In [ ]:
# 5. Duration of ISI Transitions from Zero to Non-Zero and Back to Zero

with engine.connect() as connection:
    duration_of_cycle_query = '''
    WITH transitions AS (
    SELECT 
        Latitude,
        Longitude,
        Time,
        ISI,
        LAG(ISI) OVER (PARTITION BY Latitude, Longitude ORDER BY Time) AS prev_ISI,
        LEAD(ISI) OVER (PARTITION BY Latitude, Longitude ORDER BY Time) AS next_ISI
    FROM isi_monthly_data
),
event_durations AS (
    SELECT
        Latitude,
        Longitude,
        MIN(CASE WHEN prev_ISI = 0 AND ISI > 0 THEN Time END) AS start_time,
        MAX(CASE WHEN ISI > 0 AND next_ISI = 0 THEN Time END) AS end_time
    FROM transitions
    GROUP BY Latitude, Longitude
)
SELECT
    Latitude,
    Longitude,
    start_time,
    end_time,
    DATEDIFF(end_time, start_time) AS duration_days
FROM event_durations
WHERE start_time IS NOT NULL AND end_time IS NOT NULL;
    '''
    duration_of_cycle = pd.read_sql(duration_of_cycle_query,con=connection.connection)

duration_of_cycle